In [ ]:
##### LOADING THE SN LIGHT CURVE #####

# SN TNS name
tnsname = ''

# SN discovery date
discovery_date = None

# path to directory that contains SN and control light curves
source_dir = ''

# path to save plots
output_dir = source_dir+'/bump_analysis/plots'

# filter of light curve to analyze
filt = 'o'

# MJD bin size in days of light curve to analyze
mjd_bin_size = 1.0

# define flags that define bad measurements (to filter out bad days in lc)
flags = 0x800000

##### LOADING CONTROL LIGHT CURVES #####

# number of control light curves to load
n_controls = 8

In [ ]:
import pandas as pd
import numpy as np
import sys,copy,random,math
from scipy import interpolate
from pdastro import pdastrostatsclass
from asym_gaussian import gauss2lc
from bump_detection import EfficiencyTable, Eruption, LightCurve

# plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
# default plotting styles
plt.rc('axes', titlesize = 17)
plt.rc('xtick', labelsize = 11)
plt.rc('ytick', labelsize = 11)
plt.rc('legend', fontsize = 9)
plt.rcParams['font.size'] = 11
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=['red', 'orange', 'green', 'blue', 'purple', 'magenta'])
matplotlib.rcParams['xtick.major.size'] = 6
matplotlib.rcParams['xtick.major.width'] = 1
matplotlib.rcParams['xtick.minor.size'] = 3
matplotlib.rcParams['xtick.minor.width'] = 1
matplotlib.rcParams['ytick.major.size'] = 6
matplotlib.rcParams['ytick.major.width'] = 1
matplotlib.rcParams['ytick.minor.size'] = 3
matplotlib.rcParams['ytick.minor.width'] = 1
matplotlib.rcParams['axes.linewidth'] = 1

# suppress matplotlib warnings
import warnings
warnings.simplefilter('error', RuntimeWarning)
warnings.filterwarnings("ignore")

# for storing lc and control lcs 
global lcs
lcs = {}
# for storing info about the lc
global lc_info
# for storing detected tables
global detected
detected = None
# for storing efficiencies
global efficiencies
efficiencies = None

def AandB(A,B):
    return np.intersect1d(A,B,assume_unique=False)

def AnotB(A,B):
    return np.setdiff1d(A,B)

def AorB(A,B):
    return np.union1d(A,B)

def not_AandB(A,B):
    return np.setxor1d(A,B)

# get all indices of a dataframe
def get_ix(df):
    return df.index.values

# convert flux to magnitude 
def flux2mag(flux):
    return -2.5 * np.log10(flux) + 23.9

# convert magnitude to flux
def mag2flux(mag):
    return 10 ** ((mag - 23.9) / -2.5)

def apply_gaussian(lc, gaussian_sigma, sim_gauss=False, sim_peakmjd=None, sim_appmag=None, sim_sigma=None, print_=True, flag=0x800000):
    ix = get_ix(lc.t)
    if len(ix) < 1:
        raise RuntimeError('ERROR: not enough measurements to apply simulated gaussian')
    good_ix = AandB(ix, lc.ix_unmasked('Mask', flag)) # all good pre-SN indices

    # make sure there are no lingering simulations
    dropcols=[]
    for col in ['uJysim','SNRsim','simLC','SNRsimsum']:
        if col in lc.t.columns:
            dropcols.append(col)
    if len(dropcols) > 0:
        lc.t.drop(columns=dropcols,inplace=True)
    
    lc.t.loc[ix, 'SNR'] = 0.0
    lc.t.loc[good_ix,'SNR'] = lc.t.loc[good_ix,'uJy']/lc.t.loc[good_ix,'duJy']

    # add simulated gaussian
    if sim_gauss:
        if print_:
            print(f'# Adding simulated gaussian: peak at MJD {sim_peakmjd:0.2f}; apparent magnitude {sim_appmag:0.2f}; sigma- and sigma+ of {sim_sigma:0.2f} days')
        mjds = lc.t.loc[good_ix,'MJD']
        lc.t.loc[good_ix,'uJysim'] = lc.t.loc[good_ix,'uJy']
        lc.t.loc[ix,'simLC'] = 0.0

        # get simulated gaussian flux and add to light curve flux
        simflux = gauss2lc(mjds, sim_peakmjd, sim_sigma, sim_sigma, app_mag=sim_appmag)
        lc.t.loc[good_ix,'uJysim'] += simflux
        # get the simulated lc for all MJDs
        simflux_all = gauss2lc(lc.t.loc[ix,'MJDbin'], sim_peakmjd, sim_sigma, sim_sigma, app_mag=sim_appmag)
        lc.t.loc[ix,'simLC'] += simflux_all

        # make sure all bad rows have SNRsim = 0.0 so they have no impact on the rolling SNRsum
        lc.t.loc[ix,'SNRsim'] = 0.0
        # include simflux in the SNR
        lc.t.loc[good_ix,'SNRsim'] = lc.t.loc[good_ix,'uJysim']/lc.t.loc[good_ix,'duJy']

    new_gaussian_sigma = round(gaussian_sigma/lc_info['mjdbinsize'])
    windowsize = int(6 * new_gaussian_sigma)
    halfwindowsize = int(windowsize * 0.5) + 1
    if print_:
        print(f'# Sigma: {gaussian_sigma:0.2f} days; MJD bin size: {lc_info["mjdbinsize"]:0.2f} days; sigma: {new_gaussian_sigma:0.2f} bins; window size: {windowsize} bins')

    # calculate the rolling SNR sum
    dataindices = np.array(range(len(lc.t.loc[ix])) + np.full(len(lc.t.loc[ix]), halfwindowsize))
    temp = pd.Series(np.zeros(len(lc.t.loc[ix]) + 2*halfwindowsize), name='SNR', dtype=np.float64)
    temp[dataindices] = lc.t.loc[ix,'SNR']
    SNRsum = temp.rolling(windowsize, center=True, win_type='gaussian').sum(std=new_gaussian_sigma)
    lc.t.loc[ix,'SNRsum'] = list(SNRsum[dataindices])
    # normalize it
    norm_temp = pd.Series(np.zeros(len(lc.t.loc[ix]) + 2*halfwindowsize), name='norm', dtype=np.float64)
    norm_temp[np.array(range(len(lc.t.loc[ix])) + np.full(len(lc.t.loc[ix]), halfwindowsize))] = np.ones(len(lc.t.loc[ix]))
    norm_temp_sum = norm_temp.rolling(windowsize, center=True, win_type='gaussian').sum(std=new_gaussian_sigma)
    lc.t.loc[ix,'SNRsumnorm'] = list(SNRsum.loc[dataindices] / norm_temp_sum.loc[dataindices] * max(norm_temp_sum.loc[dataindices]))

    # calculate the rolling SNR sum for SNR with simflux
    if sim_gauss:
        temp = pd.Series(np.zeros(len(lc.t.loc[ix]) + 2*halfwindowsize), name='SNRsim', dtype=np.float64)
        temp[dataindices] = lc.t.loc[ix,'SNRsim']
        SNRsimsum = temp.rolling(windowsize, center=True, win_type='gaussian').sum(std=new_gaussian_sigma)
        lc.t.loc[ix,'SNRsimsum'] = list(SNRsimsum.loc[dataindices])

    return lc

# load a single light curve
def load_lc(source_dir, control_index):
    if control_index == 0:
        filename = f'{source_dir}/{lc_info["tnsname"]}.{lc_info["filt"]}.{lc_info["mjdbinsize"]:0.2f}days.lc.txt'
        print(f'Loading light curve for SN {tnsname} at {filename} ...')
    else:
        filename = f'{source_dir}/controls/{lc_info["tnsname"]}_i{control_index:03d}.{lc_info["filt"]}.{lc_info["mjdbinsize"]:0.2f}days.lc.txt'
        print(f'Loading control light curve {control_index:03d} at {filename} ...')

    lcs[control_index] = pdastrostatsclass()
    try:
        lcs[control_index].load_spacesep(filename,delim_whitespace=True)
    except Exception as e:
        raise RuntimeError('ERROR: Could not load light curve at %s: %s' % (filename, str(e)))

# load control light curves
def load_control_lcs(source_dir, n_controls):
    for control_index in range(1,n_controls+1):
        load_lc(source_dir, control_index)

# load SN and control light curves
def load_all_lcs(source_dir, n_controls):
    load_lc(source_dir, 0)
    load_control_lcs(source_dir, n_controls)
    print('Success')

# set pre-SN (pre-discovery date) and post-SN (post-discovery date) indices
def set_discdate_ix():
    preSN_ix = lcs[0].ix_inrange(colnames=['MJDbin'], uplim=lc_info['discdate'], exclude_uplim=True)
    if len(preSN_ix) < 1:
        raise RuntimeError('ERROR: pre-SN indices must be 1 or greater')
    
    lc_info['preSN_ix'] = preSN_ix
    lc_info['postSN_ix'] = AnotB(get_ix(lcs[0].t), preSN_ix)
    
# fill lc_info
lc_info = {}
lc_info['tnsname'] = tnsname
lc_info['discdate'] = discovery_date
lc_info['filt'] = filt
lc_info['mjdbinsize'] = mjd_bin_size

# load SN and control lcs
load_all_lcs(source_dir, n_controls)

# set pre-SN and post-SN indices
set_discdate_ix()

In [ ]:
# plotting default settings

marker_size = 30
marker_edgewidth = 1.5
simbump_ls = 'dashed'
fomlimit_ls = 'dotted'

# color scheme
# FLUX
sn_flux = 'orange'
sn_flagged_flux = 'red'
ctrl_flux = 'limegreen'
select_ctrl_flux = 'darkgreen'
# FOM
sn_fom = 'magenta'
ctrl_fom = 'cornflowerblue'
select_ctrl_fom = 'mediumblue'

# Plot: cleaned original and averaged pre-SN light curves

In [ ]:
plot_mjd_ranges = True

# MJD ranges (observation seasons) with valid measurements 
mjd_ranges = [[57365,57622],
              [57762,57983], 
              [58120,58383], 
              [58494,58741-41], 
              [58822+28,59093], 
              [59184,59445], 
              [59566,59835], 
              [59901,60085]]

# plot y limits
ylims = (-300, 400)

# save plot
save = False

In [ ]:
og_lc = pdastrostatsclass()
og_lc.load(filename=f'{source_dir}/{tnsname}.o.lc.txt')

fig1, (ax1, ax2) = plt.subplots(2, constrained_layout=True)
fig1.set_figwidth(7)
fig1.set_figheight(3)

# original lc 

ax1.set_ylim(ylims[0], ylims[1])
ax1.set_xlim(og_lc.t.loc[0,'MJD'],discovery_date)
ax1.minorticks_on()
ax1.get_xaxis().set_ticks([])
ax1.tick_params(direction='in', which='both')
ax1.set_ylabel(r'Flux ($\mu$Jy)')
ax1.axhline(linewidth=1, color='k', zorder=0)

good_ix = og_lc.ix_unmasked('Mask',maskval=0x1|0x2|0x400000)
bad_ix = AnotB(og_lc.getindices(),good_ix)

# cleaned
ax1.errorbar(og_lc.t.loc[good_ix,'MJD'], og_lc.t.loc[good_ix,'uJy'], yerr=og_lc.t.loc[good_ix,'duJy'], fmt='none', ecolor=sn_flux, elinewidth=1.5, capsize=1.2, c=sn_flux, alpha=0.5, zorder=0)
ax1.scatter(og_lc.t.loc[good_ix,'MJD'], og_lc.t.loc[good_ix,'uJy'], s=marker_size, lw=marker_edgewidth, color=sn_flux, marker='o', alpha=0.5, label=f'Cleaned Measurements', zorder=0)

# flagged
ax1.errorbar(og_lc.t.loc[bad_ix,'MJD'], og_lc.t.loc[bad_ix,'uJy'], yerr=og_lc.t.loc[bad_ix,'duJy'], fmt='none', ecolor=sn_flagged_flux, elinewidth=1.5, capsize=1.2, c=sn_flagged_flux, alpha=0.5, zorder=10)
ax1.scatter(og_lc.t.loc[bad_ix,'MJD'], og_lc.t.loc[bad_ix,'uJy'], s=marker_size, lw=marker_edgewidth, facecolors='none', edgecolors=sn_flagged_flux, marker='o', alpha=0.5, label=f'Flagged Measurements', zorder=10)

ax1.legend(loc='upper right', facecolor='white', framealpha=1.0).set_zorder(100)

# averaged lc

ax2.set_ylim(ylims[0], ylims[1])
ax2.set_xlim(og_lc.t.loc[0,'MJD'],discovery_date)
ax2.minorticks_on()
ax2.tick_params(direction='in', which='both')
ax2.set_ylabel(r'Flux ($\mu$Jy)')
ax2.set_xlabel('MJD')
ax2.axhline(linewidth=1, color='k', zorder=0)

good_ix = lcs[0].ix_unmasked('Mask',maskval=flags)
bad_ix = AnotB(og_lc.getindices(),good_ix)

# cleaned
ax2.errorbar(lcs[0].t.loc[good_ix,'MJD'], lcs[0].t.loc[good_ix,'uJy'], yerr=lcs[0].t.loc[good_ix,'duJy'], fmt='none', ecolor=sn_flux, elinewidth=1.5, capsize=1.2, c=sn_flux, alpha=0.5, zorder=0)
ax2.scatter(lcs[0].t.loc[good_ix,'MJD'], lcs[0].t.loc[good_ix,'uJy'], s=marker_size, lw=marker_edgewidth, color=sn_flux, marker='o', alpha=0.5, label=f'Cleaned Measurements', zorder=0)

# flagged
ax2.errorbar(lcs[0].t.loc[bad_ix,'MJD'], lcs[0].t.loc[bad_ix,'uJy'], yerr=lcs[0].t.loc[bad_ix,'duJy'], fmt='none', ecolor=sn_flagged_flux, elinewidth=1.5, capsize=1.2, c=sn_flagged_flux, alpha=0.5, zorder=10)
ax2.scatter(lcs[0].t.loc[bad_ix,'MJD'], lcs[0].t.loc[bad_ix,'uJy'], s=marker_size, lw=marker_edgewidth, facecolors='none', edgecolors=sn_flagged_flux, marker='o', alpha=0.5, label=f'Flagged Measurements', zorder=10)

ax2.legend(loc='upper right', facecolor='white', framealpha=1.0).set_zorder(100)

if plot_mjd_ranges:
    # valid mjd ranges
    for mjd_range in mjd_ranges:
        ax1.axvline(mjd_range[0], color='magenta')
        ax1.axvline(mjd_range[1], color='magenta')

        ax2.axvline(mjd_range[0], color='magenta')
        ax2.axvline(mjd_range[1], color='magenta')

if save:
    plt.savefig(f'{output_dir}/cleaned_avg.png', dpi=200)

# Plot: pre-SN and selected control light curve

In [ ]:
# select the control light curve to plot
select_index = 1

# plot y limits
ylims = (-150, 150)

# save plot
save = False

In [ ]:
fig1, (ax1, ax2) = plt.subplots(2, constrained_layout=True)
fig1.set_figwidth(7)
fig1.set_figheight(3)

ax1.set_ylim(ylims[0], ylims[1])
ax1.set_xlim(lcs[0].t.loc[0,'MJD'],discovery_date)
ax1.minorticks_on()
ax1.get_xaxis().set_ticks([])
ax1.tick_params(direction='in', which='both')
ax1.set_ylabel(r'Flux ($\mu$Jy)')
ax1.axhline(linewidth=1, color='k', zorder=0)

# pre-SN lc flux
good_ix = lcs[0].ix_unmasked('Mask',maskval=flags)
ax1.errorbar(lcs[0].t.loc[good_ix,'MJD'], lcs[0].t.loc[good_ix,'uJy'], yerr=lcs[0].t.loc[good_ix,'duJy'], fmt='none', ecolor=sn_flux, elinewidth=1.5, capsize=1.2, c=sn_flux, alpha=0.5, zorder=20)
ax1.scatter(lcs[0].t.loc[good_ix,'MJD'], lcs[0].t.loc[good_ix,'uJy'], s=marker_size, lw=marker_edgewidth, color=sn_flux, marker='o', alpha=0.5, label=f'Cleaned Pre-SN Light Curve', zorder=20)

ax2.set_ylim(ylims[0], ylims[1])
ax2.set_xlim(lcs[0].t.loc[0,'MJD'],discovery_date)
ax2.minorticks_on()
ax2.tick_params(direction='in', which='both')
ax2.set_ylabel(r'Flux ($\mu$Jy)')
ax2.set_xlabel('MJD')
ax2.axhline(linewidth=1, color='k', zorder=0)

# control lc flux
good_ix = lcs[select_index].ix_unmasked('Mask',maskval=flags)
ax2.errorbar(lcs[select_index].t.loc[good_ix,'MJD'], lcs[select_index].t.loc[good_ix,'uJy'], yerr=lcs[select_index].t.loc[good_ix,'duJy'], fmt='none', ecolor=select_ctrl_flux, elinewidth=1.5, capsize=1.2, c=select_ctrl_flux, alpha=0.5, zorder=10)
ax2.scatter(lcs[select_index].t.loc[good_ix,'MJD'], lcs[select_index].t.loc[good_ix,'uJy'], s=marker_size, lw=marker_edgewidth, color=select_ctrl_flux, marker='o', alpha=0.5, label=f'Cleaned Control Light Curve #{select_index}', zorder=10)

if save:
    plt.savefig(output_dir+'/avg_control.png', dpi=200)

# Plot: weighted gaussian rolling sum of 3 illustratory simulated bumps

In [ ]:
# select the control light curve to add simulated bumps to
select_index = 7

# FOM detection limit
fom_limits = [3, 5]

# gaussian sigma of rolling gaussian weighted sum
gauss_sigma = 5

# number of simulated bumps to add
n_sim_gauss = 3 

# peak mjd for each simulated bump
sim_peak_mjds = [57900, 58600, 59250]

# gaussian sigma of each simulated bump
sim_gauss_sigmas = [5, 5, 5]

# peak flux of each simulated bump
sim_peaks = [10, 20, 30] 

# y coordinates for text labels
label_y = [9, 8, 13]

# save plot
save = False

In [ ]:
# apply weighted gaussian rolling sum to control lc
lcs[select_index] = apply_gaussian(lcs[select_index], gauss_sigma, print_=False)

ctrl_color = 'blue'

# plot 2
fig4, ax1 = plt.subplots(1, constrained_layout=True)
fig4.set_figwidth(5)
fig4.set_figheight(3)

ax1.set_xlim(58600-1100,58600+1200)
ax1.set_ylim(-8,18)
ax1.minorticks_on()
ax1.tick_params(direction='in', which='both')
ax1.set_ylabel(r'$\Sigma_{\rm FOM}$')
ax1.set_xlabel('MJD')
ax1.axhline(linewidth=1, color='k', zorder=0)
ax1.plot(lcs[select_index].t['MJDbin'], lcs[select_index].t['SNRsumnorm'], color=select_ctrl_fom, alpha=1, zorder=10)

# add simulated gaussians to selected control light curve
for i in range(n_sim_gauss): # for each simulated gaussian
    # add that simulated gaussian and apply gaussian weighted rolling sum
    simlc = copy.deepcopy(lcs[select_index])
    sim_appmag = -2.5 * np.log10(sim_peaks[i]) + 23.9
    simlc = apply_gaussian(simlc, gauss_sigma, sim_gauss=True, sim_peakmjd=sim_peak_mjds[i], sim_appmag=sim_appmag, sim_sigma=sim_gauss_sigmas[i])
    
    # plot it
    ax1.plot(simlc.t['MJDbin'], simlc.t['SNRsimsum'], linestyle='dashed', color=select_ctrl_fom, zorder=0)
    ax1.text(sim_peak_mjds[i]+30, label_y[i], r'$m_{peak}$ = '+f'{sim_appmag:0.2f}', fontsize=11) 

for fom_limit in fom_limits:
    ax1.axhline(fom_limit, linewidth=1, color='black', linestyle='dashed') 
    text = ax1.text(57540, fom_limit+0.7, r'$\Sigma_{\rm FOM, limit}$ = '+str(fom_limit), fontsize=11)

ax1.plot([0,1],[0,0], color=select_ctrl_fom, alpha=1, label=f'Rolling Sum')
ax1.plot([0,1],[0,0], color=select_ctrl_fom, linestyle='dashed', alpha=1, label=f'Rolling Sum with Added Bumps')
plt.legend(loc='upper left', facecolor='white', framealpha=1.0).set_zorder(100)

if save:
    plt.savefig(output_dir+'/exsimbumps.png', dpi=200)

# Get FOM limits

In [ ]:
# select the control light curve to highlight
select_index = 2

# skip certain control lcs
skip_ctr = [3]

# optionally remove bad MJD range(s)
mjd_ranges_remove = [[57365,57622]]

# sigmas of weighted gaussian rolling sums
gauss_sigmas = [5,40,80,130,200,256,300]

# plot illustratory histograms
plot = False

In [ ]:
def in_valid_season(mjd, valid_seasons):
    in_season = False
    for season in valid_seasons:
        if mjd <= season[1] and mjd >= season[0]:
            in_season = True
    return in_season

def get_valid_ix(control_index, valid_seasons=None):
    if valid_seasons is None: # all mjds valid
        return get_ix(lcs[control_index].t)
    
    valid_ix = []
    for i in range(len(lcs[control_index].t)):
        if in_valid_season(lcs[control_index].t.loc[i,'MJDbin'], valid_seasons):
            valid_ix.append(i)
    return valid_ix

def get_allfoms(gauss_sigmas, skip_ctr, valid_seasons):
    allfoms = {gauss_sigma: None for gauss_sigma in gauss_sigmas} 
    print(f'Getting all FOM for valid seasons {valid_seasons}...')
    for i in range(len(gauss_sigmas)):
        gauss_sigma = gauss_sigmas[i]
        
        allfom = pd.Series()
        for control_index in range(1, n_controls+1):
            if control_index in skip_ctr:
                continue
            
            ix = get_valid_ix(control_index, valid_seasons=valid_seasons)
            
            lcs[control_index] = apply_gaussian(lcs[control_index], gauss_sigma, print_=False)
            allfom = pd.concat([allfom, lcs[control_index].t.loc[ix,'SNRsumnorm']], ignore_index=True)

        allfoms[gauss_sigma] = allfom
    return allfoms

def get_fom_limits(gauss_sigma, allfom, plot=False):
    df = pdastrostatsclass(columns=['SNRsumnorm'])
    df.t['SNRsumnorm'] = allfom

    df.calcaverage_sigmacutloop('SNRsumnorm', Nsigma=3.0, median_firstiteration=True)
    stdev = df.statparams["stdev"]
    mean = df.statparams["mean"]
    fom_limit1 = mean + 3*stdev
    fom_limit2 = mean + 5*stdev
    print(f'T_G={gauss_sigma}: stdev={df.statparams["stdev"]:0.2f}, 3*stdev={fom_limit1:0.2f}, 5*stdev={fom_limit2:0.2f}')

    if plot:
        plt.figure(figsize=(4,2))
        plt.title(r'FOM distribution for ' + "\N{greek small letter tau}" + r"$_{G}$ = " + f'{gauss_sigma} days', fontsize=15)
        plt.hist(allfom, bins=15, color=ctrl_fom)
        plt.axvline(mean, linewidth=1.5, color='k') 
        plt.axvline(fom_limit1, linewidth=1.5, color='k', linestyle=fomlimit_ls) 
        plt.axvline(fom_limit2, linewidth=1.5, color='k', linestyle=fomlimit_ls) 
        plt.xlabel('FOM')

    return fom_limit1, fom_limit2

def get_all_fom_limits(gauss_sigmas, allfoms, plot=False):
    fom_limits = [None] * len(gauss_sigmas)
    for i in range(len(gauss_sigmas)):
        gauss_sigma = gauss_sigmas[i]
        fom_limit1, fom_limit2 = get_fom_limits(gauss_sigma, allfoms[gauss_sigma], plot=plot)
        fom_limits[i] = [round(fom_limit1, 2), round(fom_limit2, 2)]
    return fom_limits

allfoms    = get_allfoms(gauss_sigmas, skip_ctr, valid_seasons=mjd_ranges) 
allfoms_s1 = get_allfoms(gauss_sigmas, skip_ctr, valid_seasons=[mjd_range for mjd_range in mjd_ranges if mjd_range not in mjd_ranges_remove]) 
allfoms_s2 = get_allfoms(gauss_sigmas, skip_ctr, valid_seasons=mjd_ranges_remove)

fom_limits         = get_all_fom_limits(gauss_sigmas, allfoms)
print(f'FOM limits: \n{fom_limits}')

fom_limits_sample1 = get_all_fom_limits(gauss_sigmas, allfoms_s1, plot=True)
print(f'FOM limits (bad mjd range {mjd_ranges_remove} removed): \n{fom_limits_sample1}')

fom_limits_sample2 = get_all_fom_limits(gauss_sigmas, allfoms_s2, plot=True)
print(f'FOM limits (bad mjd range {mjd_ranges_remove} only): \n{fom_limits_sample2}')

# Plot: rolling sums and FOM histograms

In [ ]:
def plot_foms(gauss_sigma, skip_ctr, fom_limits, fom_limits_x, allfoms, mjd_ranges_remove=None, save=False):
    print(f'Plotting (save={save}): gauss sigma {gauss_sigma}, fom_limits {fom_limits}, mjd ranges to remove {mjd_ranges_remove}')

    # apply rolling gaussian weighted sum to pre-SN and control light curves
    for control_index in range(0, n_controls+1):
        if control_index in skip_ctr:
            continue
        lcs[control_index] = apply_gaussian(lcs[control_index], gauss_sigma, print_=False)

    fig, (ax1,ax2) = plt.subplots(1,2, gridspec_kw={'width_ratios': [4, 1]}, constrained_layout=True)
    fig.set_figwidth(5)
    fig.set_figheight(3)

    # FOM histogram
    ax2.hist(allfoms[gauss_sigma], bins=15, orientation="horizontal", color=ctrl_fom)
    ax2.set_ylim(-25, fom_limits[1]+5) # FIX 
    ax2.yaxis.set_tick_params(labelbottom=False)
    ax2.set_xlabel('Freq')
    ax2.minorticks_on()
    ax2.tick_params(direction='in', which='both')
    for fom_limit in fom_limits:
        ax2.axhline(fom_limit, linewidth=1.5, color='k', linestyle=fomlimit_ls, zorder=40) 

    ax1.set_xlabel('MJD')
    #ax1.set_xlim(57500,lc_info['discdate']-30)
    if not mjd_ranges_remove is None:
        valid_mjd_ranges = [mjd_range for mjd_range in mjd_ranges if mjd_range not in mjd_ranges_remove]
        ax1.set_xlim(valid_mjd_ranges[0][0]-20, min(valid_mjd_ranges[-1][1]+20, lc_info['discdate']))
    else:
        ax1.set_xlim(57500,lc_info['discdate'])
    ax1.set_ylim(-25, fom_limits[1]+5) # FIX 
    ax1.minorticks_on()
    ax1.tick_params(direction='in', which='both')
    ax1.set_ylabel(r'Figure of Merit ($\Sigma_{\rm FOM}$)') #('Figure of Merit (FOM)')
    ax1.axhline(linewidth=1.5, color='k', zorder=0)

    # control lcs flux
    for control_index in range(1, n_controls):
        if control_index in skip_ctr or control_index == select_index:
            continue
        ax1.plot(lcs[control_index].t['MJDbin'], lcs[control_index].t['SNRsumnorm'], color=ctrl_fom, alpha=0.3, zorder=10)
    ax1.plot([0,1],[0,0], color=ctrl_fom, linewidth=1.5,  alpha=0.6, zorder=0, label=f'{n_controls-len(skip_ctr)-1} Control Light Curves')

    # selected control lc flux
    ax1.plot(lcs[select_index].t['MJDbin'], lcs[select_index].t['SNRsumnorm'], color=select_ctrl_fom, linewidth=1.5, alpha=0.7, zorder=20, label=f'Selected Control Lightcurve #{select_index}')

    # pre-SN lc flux
    ax1.plot(lcs[0].t['MJDbin'], lcs[0].t['SNRsumnorm'], color=sn_fom, linewidth=1.5, label='Pre-SN Light Curve', zorder=40)

    for i in range(len(fom_limits)):
        fom_limit = fom_limits[i]
        if i == 0:
            text = r'$(3\sigma)$'
        else:
            text = r'$(5\sigma)$'
        ax1.axhline(fom_limit, linewidth=1.5, color='k', linestyle=fomlimit_ls) 
        ax1.text(fom_limits_x, fom_limit+1.2, r'$\Sigma_{\rm FOM, limit}$ '+text+' = '+str(fom_limit), fontsize=12, color='k')
    ax1.legend(loc='lower left', facecolor='white', framealpha=1.0).set_zorder(100)

    if save:
        plt.savefig(output_dir+'/allfoms.png', dpi=200)

def plot_all_foms(gauss_sigmas, skip_ctr, fom_limits, fom_limits_x, allfoms, mjd_ranges_remove=None):
    for i in range(len(gauss_sigmas)):
        gauss_sigma = gauss_sigmas[i]
        save = False
        if gauss_sigma == 5:
            save = True
        plot_foms(gauss_sigma, skip_ctr, fom_limits[i], fom_limits_x, allfoms, mjd_ranges_remove=mjd_ranges_remove, save=save)

plot_all_foms(gauss_sigmas, skip_ctr, fom_limits_sample1, 57900, allfoms_s1, mjd_ranges_remove=mjd_ranges_remove)

if not mjd_ranges_remove is None:
    plot_all_foms(gauss_sigmas, skip_ctr, fom_limits_sample2, 57300, allfoms_s2,
                  mjd_ranges_remove=[mjd_range for mjd_range in mjd_ranges if mjd_range not in mjd_ranges_remove])

# Efficiencies

In [ ]:
# directory where simulation detection tables and efficiency tables are stored
tables_dir = ''

# load effiency tables
load = False
load_filename_s1 = 'efficiencies_s1.txt'
load_filename_s2 = 'efficiencies_s2.txt'

# save efficiency tables
save = True
save_filename_s1 = 'efficiencies_s1.txt'
save_filename_s2 = 'efficiencies_s2.txt'

gauss_sigmas = [5,40,80,130,200,256,300]
sim_sigmas = [[2, 5, 20, 40, 80, 120], # 5
              [5, 20, 40, 80, 110, 150, 200, 250], # 40
              [5, 20, 40, 80, 110, 150, 200, 250], # 80
              [20, 40, 80, 110, 150, 200, 250, 300], # 140
              [20, 40, 80, 110, 150, 200, 250, 300], # 200
              [20, 40, 80, 110, 150, 200, 250, 300], # 256
              [20, 40, 80, 110, 150, 200, 250, 300]] # 300
peak_fluxes = [2.29, 3.22, 4.52, 6.34, 8.9, 12.5, 17.55, 24.64, 34.59, 48.56, 68.18, 95.73, 134.41, 188.71, 264.95, 371.99, 522.27, 733.27, 1029.51, 1445.44]
peak_appmags = [23.0, 22.63, 22.26, 21.89, 21.53, 21.16, 20.79, 20.42, 20.05, 19.68, 19.32, 18.95, 18.58, 18.21, 17.84, 17.47, 17.11, 16.74, 16.37, 16.0]

In [ ]:
from bump_detection import EfficiencyTable, SimDetecTable, load_sd_dict

e_s1 = EfficiencyTable(gauss_sigmas, peak_appmags, peak_fluxes, sim_sigmas)
e_s1.set_fom_limits(fom_limits_sample1)
print('Sample 1')
print(f'# Gauss sigmas: {e_s1.gauss_sigmas}')
print(f'# Sim sigmas: {e_s1.sim_sigmas}')
print(f'# FOM limits: {e_s1.fom_limits}')

e_s2 = EfficiencyTable(gauss_sigmas, peak_appmags, peak_fluxes, sim_sigmas)
e_s2.set_fom_limits(fom_limits_sample2)
print('Sample 2')
print(f'Gauss sigmas: {e_s2.gauss_sigmas}')
print(f'Sim sigmas: {e_s2.sim_sigmas}')
print(f'FOM limits: {e_s2.fom_limits}')

if load:
    print(f'Loading efficiencies from {load_filename_s1} and {load_filename_s2}...')
    e_s1.load(tables_dir, load_filename_s1) # load sample 1 efficiency table
    e_s2.load(tables_dir, load_filename_s2) # load sample 2 efficiency table
else:
    # recalculate efficiencies using FOM limits
    e_s1.reset()
    e_s2.reset()
    sd = load_sd_dict(gauss_sigmas, peak_appmags, tables_dir)
    print('Successfully loaded all simulation detection tables')
    print(f'Recalculating efficiencies...')
    e_s1.get_efficiencies(sd, mjd_ranges=[mjd_range for mjd_range in mjd_ranges if mjd_range not in mjd_ranges_remove])
    e_s2.get_efficiencies(sd, mjd_ranges=mjd_ranges_remove)

print(e_s1.t.to_string())
print(e_s2.t.to_string())

if save:
    e_s1.save(tables_dir, filename=save_filename_s1)
    e_s2.save(tables_dir, filename=save_filename_s2)

In [ ]:
linestyles = ['solid', 'dashed']
colors = ['red', 'orange', 'green', 'blue', 'magenta']

def plot_efficiencies(e, gauss_sigma, thresholds=False):
    fig1, ax1 = plt.subplots(1, constrained_layout=True)
    fig1.set_figwidth(7)
    fig1.set_figheight(3.5)
        
    ax1.set_title(r'Efficiencies for ' +  "\N{greek small letter tau}" + r"$_{G}$ = " + f'{gauss_sigma} days', pad=10)
    ax1.axhline(linewidth=1,color='k')
    ax1.set_ylabel('Efficiency (%)', fontsize=14)

    for k in range(len(e.sim_sigmas[i])):
        sim_sigma = e.sim_sigmas[i][k]
        for j in range(len(e.fom_limits[gauss_sigma])):
            fom_limit = e.fom_limits[gauss_sigma][j] 
            if sim_sigma is None:
                sim_sigma = 2.8
                subset = e.get_subset(gauss_sigma=gauss_sigma, sim_sigma=sim_sigma, erup=True, fom_limit=fom_limit)
            else:
                subset = e.get_subset(gauss_sigma=gauss_sigma, sim_sigma=sim_sigma, fom_limit=fom_limit)
                        
            label = "\N{greek small letter tau}" + r"$'_{G}$ = " + f'{sim_sigma} days' +', $FOM_{limit}$ = '+str(fom_limit)
            color = colors[k]
            ax1.plot(subset['peak_appmag'], subset[f'pct_detec_{fom_limit}'], linestyle=linestyles[j], label=label, color=color)

    ax1.legend(loc='upper left', bbox_to_anchor=(1,1.05), fontsize=10)
    ax1.set_xlabel(r'$m_{peak}$ (app mag)')
    ax1.set_xlim(18,22)

    if thresholds:
        ax1.axhline(80, color='black')
        ax1.axhline(50, color='black')

    # abs mag
    ax3 = ax1.twiny()
    ax3.set_xticks(ax1.get_xticks() )
    ax3.set_xbound(ax1.get_xbound())
    ax3.set_xticklabels([round(x-29.04) for x in ax1.get_xticks()])
    ax3.set_xlabel("$m_{peak}$ (abs mag)")

def plot_all_efficiencies(e, thresholds=False):
    for i in range(len(e.gauss_sigmas)):
        gauss_sigma = e.gauss_sigmas[i]
        plot_efficiencies(e, gauss_sigma, thresholds=thresholds)

plot_all_efficiencies(e_s1)

# Plot: adding simulated gaussian bumps to a control light curve

In [ ]:
# select the control light curve to add the simulated bump to
select_index = 1

# sigmas of weighted gaussian rolling sums
gauss_sigmas = [5, 20, 40, 70]

# peak mjds of simulated bumps
sim_peak_mjds = [58015, 58015, 58015, 58015]

# gaussian sigmas of simulated bumps
sim_gauss_sigmas = [5, 20, 40, 70]

# peak fluxes of simulated bumps
sim_peaks = [30, 30, 30, 30]

# abs value of FOM y limits
ylimits_fom = [30, 40, 100, 200]

# abs value of flux y limits
ylimits_flux = [40, 50, 50, 50]

# FOM text label x position
fom_limit_x = [57990, 57910, 57990, 57990]

# save plot
save = True

In [ ]:
plt.rc('xtick', labelsize = 13)
plt.rc('ytick', labelsize = 13)
plt.rc('legend', fontsize = 9)
plt.rcParams['font.size'] = 13

matplotlib.rcParams['xtick.major.width'] = 1.5
matplotlib.rcParams['xtick.minor.width'] = 1.5
matplotlib.rcParams['ytick.major.width'] = 1.5
matplotlib.rcParams['ytick.minor.width'] = 1.5
matplotlib.rcParams['axes.linewidth'] = 1.5

In [ ]:
def plot_sim_bump(fom_limits, simlc, sim_gauss_sigma, sim_peak_flux, sim_peak_mjd, xlims=None, ylims_flux=None, ylims_fom=None, fom_limit_x=None, save=False):
    fig2, (ax1, ax3) = plt.subplots(2, constrained_layout=True)
    fig2.set_figwidth(5)
    fig2.set_figheight(5)
    fig2.tight_layout()

    # PANEL 1

    ax1.set_ylabel(r'Flux (µJy)')
    ax1.axhline(linewidth=1.5,color='k')
    ax1.tick_params(axis='y')
    ax1.minorticks_on()
    ax1.tick_params(direction='in', which='both')
    ax1.get_xaxis().set_ticks([])
    ax1.set_ylim(ylims_flux[0], ylims_flux[1])
    ax1.set_xlim(xlims[0], xlims[1])

    # control lc flux
    good_ix = lcs[select_index].ix_unmasked('Mask',maskval=flags)
    ax1.errorbar(lcs[select_index].t.loc[good_ix,'MJD'], lcs[select_index].t.loc[good_ix,'uJy'], yerr=lcs[select_index].t.loc[good_ix,'duJy'], fmt='none', elinewidth=1.5, capsize=1.2, c=select_ctrl_flux, alpha=0.5, zorder=20)
    ax1.scatter(lcs[select_index].t.loc[good_ix,'MJD'], lcs[select_index].t.loc[good_ix,'uJy'], s=marker_size, lw=marker_edgewidth, color=select_ctrl_flux, marker='o', alpha=0.5, zorder=20)

    # control lc FOM
    ax2 = ax1.twinx()
    ax2.minorticks_on()
    ax2.tick_params(axis='y', direction='in', which='both')
    ax2.set_ylim(ylims_fom[0], ylims_fom[1])
    ax2.set_xlim(xlims[0], xlims[1]) 
    ax2.set_ylabel(r'$\Sigma_{\rm FOM}$')
    ax2.scatter([0,1],[0,0], color=select_ctrl_flux, alpha=1, s=marker_size, lw=marker_edgewidth, marker='o', zorder=0, label=f'Light Curve')
    ax2.plot(lcs[select_index].t['MJDbin'], lcs[select_index].t['SNRsumnorm'], color=select_ctrl_fom, linewidth=1.5, alpha=1, label='Rolling Sum')
    plt.legend(loc='lower left', facecolor='white', framealpha=1.0).set_zorder(100)

    # PANEL 2
    
    ax3.set_xlabel('MJD')
    ax3.set_ylabel(r'Flux (µJy)')
    ax3.minorticks_on()
    ax3.tick_params(axis='y', direction='in', which='both')
    #ax3.xaxis.set_major_locator(ticker.LinearLocator(6))
    ax3.xaxis.set_minor_locator(ticker.LinearLocator(31))
    ax3.xaxis.set_major_locator(ticker.MaxNLocator(nbins=5, integer=True))
    ax3.axhline(linewidth=1.5,color='k')

    # sim control lc flux
    ax3.set_ylim(ylims_flux[0], ylims_flux[1])
    ax3.set_xlim(xlims[0], xlims[1])
    ax3.errorbar(simlc.t.loc[good_ix,'MJD'], simlc.t.loc[good_ix,'uJysim'], yerr=simlc.t.loc[good_ix,'duJy'], fmt='none', elinewidth=1.5, capsize=1.2, c=select_ctrl_flux, alpha=0.5, zorder=10)
    ax3.scatter(simlc.t.loc[good_ix,'MJD'], simlc.t.loc[good_ix,'uJysim'], s=marker_size, lw=marker_edgewidth, color=select_ctrl_flux, marker='o', alpha=0.5, zorder=10)
    
    # sim gaussian flux
    def my_gauss(x, sigma=sim_gauss_sigma, h=sim_peak_flux, mid=sim_peak_mjd):
        variance = math.pow(sigma, 2)
        return h * math.exp(-math.pow(x-mid, 2)/(2*variance))
    x = np.linspace(sim_peak_mjd - 3*sim_gauss_sigma, sim_peak_mjd + 3*sim_gauss_sigma, 100)
    y = [my_gauss(xi) for xi in x]
    ax3.plot(x, y, color=select_ctrl_flux, linestyle=simbump_ls, linewidth=1.5, zorder=20)

    # sim control lc FOM
    ax4 = ax3.twinx()
    ax4.set_ylabel(r'$\Sigma_{\rm FOM}$')
    ax4.minorticks_on()
    ax4.tick_params(axis='y', direction='in', which='both')
    ax4.set_ylim(ylims_fom[0], ylims_fom[1])
    ax4.set_xlim(xlims[0], xlims[1])
    ax4.plot([0,1],[0,0], color=select_ctrl_flux, linestyle=simbump_ls, linewidth=1.5, alpha=1, label=f'Simulated Bump')
    ax4.scatter([0,1],[0,0], color=select_ctrl_flux, s=marker_size, lw=marker_edgewidth, alpha=1, marker='o', label=f'Light Curve')
    ax4.plot(simlc.t['MJDbin'], simlc.t['SNRsimsum'], color=select_ctrl_fom, linewidth=1.5, alpha=1, label=f'Rolling Sum')#label=f'Weighted Gaussian Rolling Sum of \nControl Light Curve with Added Bump')
    plt.legend(loc='lower left', facecolor='white', framealpha=1.0).set_zorder(100)

    ax2.axhline(fom_limits[0], linewidth=1.5, color=select_ctrl_fom, linestyle=fomlimit_ls)
    ax2.text(fom_limit_x, fom_limits[0]+3, r'$\Sigma_{\rm FOM, limit}$ = '+str(fom_limits[0]), fontsize=12, family='Times New Roman', color=select_ctrl_fom)
    ax4.axhline(fom_limits[0], linewidth=1.5, color=select_ctrl_fom, linestyle=fomlimit_ls)
    ax4.text(fom_limit_x, fom_limits[0]+3, r'$\Sigma_{\rm FOM, limit}$ = '+str(fom_limits[0]), fontsize=12, family='Times New Roman', color=select_ctrl_fom)

    if save:
        plt.savefig(output_dir+'/simbump.png', dpi=200)

def plot_sim_bumps(e, gauss_sigmas):
    for i in range(len(gauss_sigmas)):
        gauss_sigma = gauss_sigmas[i]
        fom_limits = e.fom_limits[gauss_sigma]

        sim_peak_mjd = sim_peak_mjds[i]
        sim_gauss_sigma = sim_gauss_sigmas[i]
        sim_peak_flux = sim_peaks[i]

        # apply weighted gaussian rolling sum to control lc
        lcs[select_index] = apply_gaussian(lcs[select_index], gauss_sigma, print_=False)

        # apply weighted gaussian rolling sum to pre-SN lc
        lcs[0] = apply_gaussian(lcs[0], gauss_sigma, print_=False)

        # add simulated gaussian and apply gaussian weighted rolling sum
        simlc = copy.deepcopy(lcs[select_index])
        sim_peak_mag = -2.5 * np.log10(sim_peak_flux) + 23.9
        simlc = apply_gaussian(simlc, gauss_sigma, sim_gauss=True, sim_peakmjd=sim_peak_mjd, sim_appmag=sim_peak_mag, sim_sigma=sim_gauss_sigma)

        save = False
        if gauss_sigma == 5:
            save = True

        plot_sim_bump(fom_limits, 
                      simlc, 
                      sim_gauss_sigma, 
                      sim_peak_flux, 
                      sim_peak_mjd, 
                      xlims=(sim_peak_mjd-sim_gauss_sigma*5.5, sim_peak_mjd+sim_gauss_sigma*5.5), 
                      ylims_flux=(-ylimits_flux[i], ylimits_flux[i]), 
                      ylims_fom=(-ylimits_fom[i], ylimits_fom[i]), 
                      fom_limit_x=fom_limit_x[i],
                      save=save)
        
plot_sim_bumps(e_s1, gauss_sigmas)

# Plot: simulated bump across kernel sizes

In [ ]:
sim_sigmas = [15, 25, 50, 100]
sim_peak_mags = [20, 21, 21.5, 22]

gauss_sigmas = [5, 40, 80, 256]
ylimits_flux = [50, 50, 50, 50]
ylimits_fom = [12, 33, 40, 60]
fom_limits_x = [58100, 58100, 58100, 58100]

plot_gauss = True

plt.rc('xtick', labelsize = 13)
plt.rc('ytick', labelsize = 13)
plt.rc('legend', fontsize = 9)
plt.rc('axes', titlesize=15)
plt.rcParams['font.size'] = 13
matplotlib.rcParams['xtick.major.width'] = 1.5
matplotlib.rcParams['xtick.minor.width'] = 1.5
matplotlib.rcParams['ytick.major.width'] = 1.5
matplotlib.rcParams['ytick.minor.width'] = 1.5
matplotlib.rcParams['axes.linewidth'] = 1.5

In [ ]:
def plot_sim_bump2(gauss_sigmas, fom_limits, sim_sigma, sim_peak_mag, sim_peak_mjd, xlims, ylims_flux, ylims_fom, fom_limits_x, plot_gauss=False):
    fig, axs = plt.subplots(2, 2, constrained_layout=True)
    fig.set_figwidth(10)
    fig.set_figheight(7)
    fig.suptitle(r'$\sigma_{sim}$ = '+f'{sim_sigma} days; '+r'$m_{peak}$ = '+f'{sim_peak_mag} mag')
    #fig.tight_layout()

    print(f'\nGauss sigmas: {gauss_sigmas}')
    print(f'FOM limits: {fom_limits}')
    print(f'Sim bump: sigma={sim_sigma}, peak mag={sim_peak_mag}, peak MJD={sim_peak_mjd}')

    for i in range(len(gauss_sigmas)):
        lcs[select_index] = apply_gaussian(lcs[select_index], gauss_sigmas[i], print_=False)
        simlc = copy.deepcopy(lcs[select_index])
        simlc = apply_gaussian(simlc, gauss_sigmas[i], sim_gauss=True, sim_peakmjd=sim_peak_mjd, sim_appmag=sim_peak_mag, sim_sigma=sim_sigma, print_=False)

        ax1 = axs.reshape(-1)[i]
        ax1.tick_params(axis='y')
        ax1.minorticks_on()
        ax1.tick_params(direction='in', which='both')
        if i==0 or i==1:
            ax1.get_xaxis().set_ticks([])
        else:
            ax1.set_xlabel('MJD')
        ax1.set_ylabel(r'Flux (µJy)')
        ax1.axhline(linewidth=1.5,color='k')
        ax1.set_title(r'$\sigma_{kern} = $'+f'{gauss_sigmas[i]} days')
        ax1.set_xlim(xlims[0],xlims[1])
        ax1.set_ylim(-ylims_flux[i], ylims_flux[i])

        # control lc flux
        good_ix = simlc.ix_unmasked('Mask',maskval=flags)
        ax1.errorbar(simlc.t.loc[good_ix,'MJD'], simlc.t.loc[good_ix,'uJysim'], yerr=simlc.t.loc[good_ix,'duJy'], fmt='none', elinewidth=1.5, capsize=1.2, c=select_ctrl_flux, alpha=0.5, zorder=10)
        ax1.scatter(simlc.t.loc[good_ix,'MJD'], simlc.t.loc[good_ix,'uJysim'], s=marker_size, lw=marker_edgewidth, color=select_ctrl_flux, marker='o', alpha=0.5, zorder=10)

        # sim gaussian flux
        if plot_gauss:
            def my_gauss(x, sigma=sim_sigma, h=mag2flux(sim_peak_mag), mid=sim_peak_mjd):
                variance = math.pow(sigma, 2)
                return h * math.exp(-math.pow(x-mid, 2)/(2*variance))
            x = np.linspace(sim_peak_mjd - 3*sim_sigma, sim_peak_mjd + 3*sim_sigma, 100)
            y = [my_gauss(xi) for xi in x]
            ax1.plot(x, y, color=select_ctrl_flux, linestyle=simbump_ls, linewidth=1.5, zorder=20) # color=select_ctrl_flux

        # sim control lc FOM
        ax2 = ax1.twinx()
        ax2.set_ylabel(r'$\Sigma_{\rm FOM}$')
        ax2.minorticks_on()
        ax2.tick_params(axis='y', direction='in', which='both')
        ax2.set_ylim(-ylims_fom[i], ylims_fom[i])
        ax2.set_xlim(xlims[0], xlims[1])
        if plot_gauss:
            ax2.plot([0,1],[0,0], color=select_ctrl_flux, linestyle=simbump_ls, linewidth=1.5, alpha=1, label=f'Simulated Bump')
        ax2.scatter([0,1],[0,0], color=select_ctrl_flux, s=marker_size, lw=marker_edgewidth, alpha=1, marker='o', label=f'Simulated Light Curve')
               
        ax2.plot(lcs[select_index].t['MJDbin'], lcs[select_index].t['SNRsumnorm'], color='red', linewidth=1.5, alpha=1, label=f'Rolling Sum')#label=f'Weighted Gaussian Rolling Sum of \nControl Light Curve with Added Bump')
        ax2.plot(simlc.t['MJDbin'], simlc.t['SNRsimsum'], color=select_ctrl_fom, linewidth=1.5, alpha=1, label=f'Simulated Rolling Sum')#label=f'Weighted Gaussian Rolling Sum of \nControl Light Curve with Added Bump')
        
        # FOM limits
        for j in range(len(fom_limits[gauss_sigmas[i]])):
            fom_limit = fom_limits[gauss_sigmas[i]][j]
            if j == 0:
                text = r'$(3\sigma)$'
            else:
                text = r'$(5\sigma)$'
            ax2.axhline(fom_limit, linewidth=1.5, color='k', linestyle=fomlimit_ls) 
            ax2.text(fom_limits_x[i], fom_limit+1.2, r'$\Sigma_{\rm FOM, limit}$ '+text+' = '+str(fom_limit), fontsize=12, color=select_ctrl_fom)
                
        plt.legend(loc='lower left', facecolor='white', framealpha=1.0).set_zorder(100)

def plot_sim_bumps2(sim_sigmas, sim_peak_mags, plot_gauss=False):
    print(f'Sim  sigmas: {sim_sigmas}')
    print(f'Peak appmags: {sim_peak_mags}')
    for i in range(len(sim_sigmas)):
        plot_sim_bump2(gauss_sigmas, 
                       e_s1.fom_limits, 
                       sim_sigmas[i], 
                       sim_peak_mags[i], 
                       58600,
                       (58600-100*5.5, 58600+100*5.5), 
                       ylimits_flux, 
                       ylimits_fom, 
                       fom_limits_x,
                       plot_gauss=plot_gauss)
        
plot_sim_bumps2(sim_sigmas, sim_peak_mags, plot_gauss=plot_gauss)

# Magnitude thresholds for 80% and 50% efficiency

In [ ]:
# save plot
save = True

In [ ]:
def get_mag_threshold(x, y, percent):
    lx = x.to_list()
    lx.reverse()
    ly = y.to_list()
    ly.reverse()

    ly_reduced = np.array(ly) - percent
    freduced = interpolate.UnivariateSpline(lx, ly_reduced, s=0)
    roots = freduced.roots()
    if len(roots) > 1:
        print(f'WARNING: Found more than one root {roots}; returning NaN')
        return np.nan
    if len(roots) < 1:
        return np.nan
    return roots[0]

def get_mag_threshold_table(e, save=False):
    mt = pd.DataFrame(columns=['gauss_sigma', 'sim_sigma', 'fom_limit', 'mag_threshold_80', 'mag_threshold_50'])

    row = 0
    for i in range(len(e.gauss_sigmas)):
        gauss_sigma = e.gauss_sigmas[i]

        for k in range(len(e.sim_sigmas[i])):
            sim_sigma = e.sim_sigmas[i][k]
            for j in range(len(e.fom_limits[gauss_sigma])):
                fom_limit = e.fom_limits[gauss_sigma][j] 
                if sim_sigma is None:
                    sim_sigma = 2.8
                    subset = e.get_subset(gauss_sigma=gauss_sigma, sim_sigma=sim_sigma, erup=True, fom_limit=fom_limit)
                else:
                    subset = e.get_subset(gauss_sigma=gauss_sigma, sim_sigma=sim_sigma, fom_limit=fom_limit)

                mag_threshold_80 = get_mag_threshold(subset['peak_appmag'], subset[f'pct_detec_{fom_limit}'], 80)
                mag_threshold_50 = get_mag_threshold(subset['peak_appmag'], subset[f'pct_detec_{fom_limit}'], 50)
                mt.loc[row] = {'gauss_sigma':gauss_sigma, 'sim_sigma':sim_sigma, 'fom_limit':fom_limit, 'mag_threshold_80':mag_threshold_80, 'mag_threshold_50':mag_threshold_50}
                row += 1

    print(mt)
    if save:
        mt.to_string(f'{tables_dir}/mag_thresholds.txt')
    return mt

mt_s1 = get_mag_threshold_table(e_s1, save=True)
mt_s2 = get_mag_threshold_table(e_s2)

In [ ]:
def plot_mag_thresholds(e, mt, ylims=None, save=False):
    for percent_efficiency in [50, 80]:
        fig, ax1 = plt.subplots(1, constrained_layout=True)
        fig.set_figwidth(5)
        fig.set_figheight(3)

        ax1.set_xlabel(r'$\sigma_{\rm sim}$ (days)')
        if not ylims is None:
            ax1.set_ylim(ylims[0],ylims[1])
        ax1.minorticks_on()
        ax1.tick_params(direction='in', which='both')
        ax1.set_ylabel(f'Apparent magnitude threshold \nfor {percent_efficiency}% efficiency (mag)')
        ax1.axhline(linewidth=1.5, color='k', zorder=0)

        for i in range(len(gauss_sigmas)):
            gauss_sigma = gauss_sigmas[i]
            gauss_sigma_ix = mt[mt['gauss_sigma'] == gauss_sigma].index

            fom_limit = e.fom_limits[gauss_sigma][0] # 3-sigma FOM limit
            fom_limit_ix = mt[mt['fom_limit'] == fom_limit].index

            ix = AandB(gauss_sigma_ix,fom_limit_ix)
            #print(mt.loc[ix,'sim_sigma'], mt.loc[ix,f'mag_threshold_{percent_efficiency}'])
            ax1.scatter(mt.loc[ix,'sim_sigma'], mt.loc[ix,f'mag_threshold_{percent_efficiency}'], s=marker_size, marker='o', label=r'$\sigma_{\rm kernel}$'+f' = {gauss_sigma}')
            ax1.plot(mt.loc[ix,'sim_sigma'], mt.loc[ix,f'mag_threshold_{percent_efficiency}'])

        plt.legend(loc='lower right', facecolor='white', framealpha=1.0).set_zorder(100)
        if save:
            plt.savefig(output_dir+f'/mag_thresholds_{percent_efficiency}.png', dpi=200)

plot_mag_thresholds(e_s1, mt_s1, ylims=(17, 22.2), save=True)
plot_mag_thresholds(e_s2, mt_s2, ylims=(15, 22.2))

# Contamination

In [ ]:
def get_contamination(control_index, gauss_sigma, fom_limit, mjd_ranges):
    lcs[control_index] = apply_gaussian(lcs[control_index], gauss_sigma, print_=False)

    # find any triggers above the fom limit
    count = 0
    mjds = []
    above_lim = False
    for k in range(len(lcs[control_index].t)):
        if lcs[control_index].t.loc[k, 'SNRsumnorm'] > fom_limit:
            if not above_lim and in_valid_season(lcs[control_index].t.loc[k, 'MJDbin'], mjd_ranges):
                mjd = lcs[control_index].t.loc[k, "MJDbin"]
                if control_index > 0: # control lc
                    mjds.append(mjd)
                    count += 1
                elif mjd < discovery_date: # MJD must be in pre-SN lc
                    mjds.append(mjd)
                    count += 1
            above_lim = True
        else:
            above_lim = False
    
    if len(mjds) > 0:
        print(f'gauss sigma {gauss_sigma}, fom limit {fom_limit}, control index {control_index}: {count} trigger(s) at mjds {mjds}')
    return count

def get_contamination_table(e, mjd_ranges, skip_ctr=None, save=False):
    n_foms = 0
    for _ in e.fom_limits:
        n_foms += len(e.fom_limits[_])

    contam = pdastrostatsclass()
    contam.t['gauss_sigma'] = [np.nan] * n_foms
    contam.t['fom'] = np.nan
    contam.t['n_controls_triggered'] = 0
    contam.t['percent_controls_triggered'] = np.nan
    contam.t['n_triggers'] = 0
    for control_index in range(1,n_controls+1):
        if not control_index in skip_ctr:
            contam.t[f'n_triggers_{control_index:02d}'] = 0

    contam_index = 0
    # for each gaussian sigma
    for i in range(len(e.gauss_sigmas)):
        gauss_sigma = e.gauss_sigmas[i]

        # for each fom limit
        for j in range(len(e.fom_limits[gauss_sigma])):
            fom_limit = e.fom_limits[gauss_sigma][j]
            contam.t.loc[contam_index, 'gauss_sigma'] = gauss_sigma
            contam.t.loc[contam_index, 'fom'] = fom_limit

            # for SN lc and each control lc
            for control_index in range(0,n_controls+1):
                if control_index in skip_ctr:
                    continue
                n_triggers = get_contamination(control_index, gauss_sigma, fom_limit, mjd_ranges)
                contam.t.loc[contam_index, f'n_triggers_{control_index:02d}'] = n_triggers
            
                if control_index > 0 and n_triggers > 0:
                    contam.t.loc[contam_index, 'n_controls_triggered'] += 1
                    contam.t.loc[contam_index, 'n_triggers'] += n_triggers
            
            contam.t.loc[contam_index,'percent_controls_triggered'] = 100 * contam.t.loc[contam_index,'n_controls_triggered']/(n_controls-len(skip_ctr))
            contam_index += 1

    contam.write()
    if save:
        contam.write(tables_dir+'/contamination.txt') # save
    return contam

contam_s1 = get_contamination_table(e_s1, 
                                    mjd_ranges=[mjd_range for mjd_range in mjd_ranges if mjd_range not in mjd_ranges_remove], 
                                    skip_ctr=skip_ctr, save=True)
contam_s2 = get_contamination_table(e_s2, mjd_ranges=mjd_ranges_remove, skip_ctr=skip_ctr)

In [ ]:
# save plot
save = True

In [ ]:
def plot_contamination(e, contam, save=False):
    fom_limit1_ix = list(np.arange(0, len(e.gauss_sigmas)*2, 2))
    fom_limit2_ix = list(np.arange(1, len(e.gauss_sigmas)*2, 2))
    
    fig1, ax1 = plt.subplots(1, constrained_layout=True)
    fig1.set_figwidth(5)
    fig1.set_figheight(3)
    ax1.minorticks_on()
    ax1.tick_params(direction='in', which='both')
    ax1.set_xlabel(r'$\sigma_{kernel}$')
    ax1.set_ylabel(f'% controls triggered')

    ax1.scatter(contam.t.loc[fom_limit1_ix, 'gauss_sigma'], contam.t.loc[fom_limit1_ix, 'percent_controls_triggered'], 
            color='r',marker='o', label=r'$3\sigma\: \Sigma_{\rm FOM, limit}$')
    ax1.plot(contam.t.loc[fom_limit1_ix, 'gauss_sigma'], contam.t.loc[fom_limit1_ix, 'percent_controls_triggered'], color='r')
    ax1.scatter(contam.t.loc[fom_limit2_ix, 'gauss_sigma'], contam.t.loc[fom_limit2_ix, 'percent_controls_triggered'], 
                color='orange',marker='o', label=r'$5\sigma\: \Sigma_{\rm FOM, limit}$')
    ax1.plot(contam.t.loc[fom_limit2_ix, 'gauss_sigma'], contam.t.loc[fom_limit2_ix, 'percent_controls_triggered'], color='orange')

    ax1.legend()

    if save:
        plt.savefig(output_dir+f'/contam_percent_controls_triggered.png', dpi=200)

    fig2, ax1 = plt.subplots(1, constrained_layout=True)
    fig2.set_figwidth(5)
    fig2.set_figheight(3)
    ax1.minorticks_on()
    ax1.tick_params(direction='in', which='both')
    ax1.set_xlabel(r'$\sigma_{kernel}$')
    ax1.set_ylabel('# triggers')

    ax1.scatter(contam.t.loc[fom_limit1_ix, 'gauss_sigma'], contam.t.loc[fom_limit1_ix, 'n_triggers'], 
                color='r',marker='o', label=r'$3\sigma\: \Sigma_{\rm FOM, limit}$')
    ax1.plot(contam.t.loc[fom_limit1_ix, 'gauss_sigma'], contam.t.loc[fom_limit1_ix, 'n_triggers'], color='r')
    ax1.scatter(contam.t.loc[fom_limit2_ix, 'gauss_sigma'], contam.t.loc[fom_limit2_ix, 'n_triggers'], 
                color='orange',marker='o', label=r'$5\sigma\: \Sigma_{\rm FOM, limit}$')
    ax1.plot(contam.t.loc[fom_limit2_ix, 'gauss_sigma'], contam.t.loc[fom_limit2_ix, 'n_triggers'], color='orange')

    ax1.legend()

    if save:
        plt.savefig(output_dir+f'/contam_n_triggers.png', dpi=200)

plot_contamination(e_s1, contam_s1, save=True)
plot_contamination(e_s2, contam_s2)